# We're scraping

Probably going to just scrape it all. 

See [baseball-scraper](https://pypi.org/project/baseball-scraper/) for examples.

In [7]:
from baseball_scraper import statcast
from baseball_scraper import playerid_lookup

import pandas as pd 
import datetime 

In [8]:
##What happens if I ask for a date on which no games are played... 
##Like I write a function and uh-oh, all-star break included...

##Looking for a date pre statcast gives a table with lots of NaN values. 
#pre_stat_scrape = statcast(start_dt='2010-08-08', verbose=True)

##Looking for a date on which no games were played gives a dataframe of shape 0x90
#january_scrape = statcast(start_dt='2018-01-01', verbose=True)




In [9]:
start = datetime.date(2020,7,23).strftime("%Y-%m-%d")
end = datetime.date(2020,8,18).strftime("%Y-%m-%d")##This is the world's third hardest way to write '2020-08-18', but you're maybe using it later.
start

'2020-07-23'

In [10]:
## Scrapes time slices based on a date range. Returns a list of successfully scraped time slices.

def big_scrape(start,end):
    '''
    Returns a DataFrame of all the successfull scraped time slices in range start, end. 
    
    If an exception is raised, prints a brief report, stops scraping, and returns
    all the successfully scraped slices.
    '''
    scrapes = [] ##
    while True:
        try:
            temp = statcast(start.strftime("%Y-%m-%d"), (start+datetime.timedelta(days=4)).strftime("%Y-%m-%d"))
        except ParserError:
            print("ParserError failure at {}".format(start.strftime("%Y-%m-%d")))
            break
        except:
            print("Unspecified failure at {}".format(start.strftime("%Y-%m-%d")))
            break
        scrapes.append(temp)
        start = start+datetime.timedelta(days=5)
        if start > end:
            break
    return pd.concat(scrapes)

In [11]:
## Creates all the .csv files.

no_really = False ##Just prevents accidentally running this cell if you have already run it and don't need to do it again.
                  ##These files are very big (300+MB) and take a long time to scrape.

if no_really:
    for dt in [2019,2018,2017,2016]:
        start = datetime.date(dt,3,20)
        end = datetime.date(dt,10,1)
        scrape = big_scrape(start,end)
        scrape.to_csv('data/statcast_dumps/statcast'+str(dt)+'.csv')
        print("Finished with statcast year {}.".format(dt))
        del scrape
else:
    print("Skipping the season scrape. To scrape, set no_really to True")


Skipping the season scrape. To scrape, set no_really to True


In [15]:
## updating the current year efficiently

def update_statcast(target_csv_name):
    '''
    Scapes any statcast data after the last game_date 
    in the year of the target file and adds it, returning a DataFrame that concatenates the
    new data with the old.
    
    '''
    out = pd.read_csv(target_csv_name)
    start = pd.Timestamp(out['game_date'].sort_values().iloc[-1])
    end = datetime.datetime.now()
    new = big_scrape(start, end)
    out = pd.concat([out,new])
    return out




In [16]:
csv_2020 = 'data/statcast_dumps/statcast2020.csv'
sc_2020 = update_statcast(csv_2020)

In [17]:
sc_2020.to_csv(csv_2020)